In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys; 
#sys.path.append('/home/mitch/school/data/mexico_enigh/src/')
sys.path.append('../src')
sys.path.append('/home/mitch/school/data/mexico_enigh/dicts/')
sys.path.append('/home/mitch/util/python/')
import json_utils

import utils
import os

from simpledbf import Dbf5

In [75]:
raw  = '/home/mitch/Dropbox/data/mexico_enigh/raw/'
interim = '/home/mitch/Dropbox/data/mexico_enigh/interim2/'
dicts = '/home/mitch/github/ENIGH/dicts/'

In [76]:
cpidir = '/home/mitch/Dropbox/data/cpi/'
os.chdir(cpidir)
cpi = pd.read_csv('cpi_annual.csv')
cpi = cpi.rename(columns = {'time':'Y', 'PCPI_IX':'cpi'})
cpi = cpi.query('country == "MX"')
cpi = cpi.drop(columns=['Unnamed: 0', 'country'])

In [77]:
os.chdir(dicts)
rename_main = json_utils.load_json('rename_main.json')
rename_population = json_utils.load_json('rename_population.json')
rename_income = json_utils.load_json('rename_income.json')
rename_location = json_utils.load_json('states.json')
hhm_ids = json_utils.load_json('hhm_ids.json')
education = json_utils.load_json('education.json')
print(rename_main)
print(rename_population)

{'folio': 'hhid', 'folioviv': 'hhid', 'ubica_geo': 'location', 'factor': 'weight', 'hog': 'weight', 'estrato': 'location_size', 'gastot': 'consumption_total', 'gascor': 'consumption_current', 'alimentos': 'consumption_food', 'vestido': 'consumption_clothing', 'vestido_c': 'consumption_clothing_c', 'salud': 'consumption_health', 'limpieza': 'consumption_house', 'transporte': 'consumption_transportation', 'educacion': 'consumption_education', 'personal': 'consumption_personal', 'ingtot': 'income_total', 'ingcor': 'income_current', 'trabajo': 'income_salary', 'negocio': 'income_business', 'rentas': 'income_rental', 'transfer': 'income_transfers', 'otros': 'income_other', 'pertot': 'financial_income', 'eromon': 'financial_expenditures', 'tam_hog': 'hh_size'}
{'folio': 'hhid', 'folioviv': 'hhid', 'paren': 'relationship_to_hhm', 'parentesco': 'relationship_to_hhm', 'sexo': 'sex', 'edad': 'age', 'tipo_esc': 'type_of_education', 'n_instr141': 'education', 'n_instr161': 'education', 'ed_formal'

In [78]:
year = '1994'
os.chdir(raw + year)
main = Dbf5('main.dbf').to_dataframe()
population = Dbf5('population.dbf').to_dataframe()

main.columns = [x.lower() for x in main.columns]
population.columns = [x.lower() for x in population.columns]

In [79]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12815 entries, 0 to 12814
Data columns (total 49 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   folio       12815 non-null  object 
 1   ubica_geo   12815 non-null  object 
 2   hog         12815 non-null  float64
 3   estrato     12815 non-null  object 
 4   ingmon      12815 non-null  float64
 5   permon      12815 non-null  float64
 6   gasnom      12815 non-null  float64
 7   eronom      12815 non-null  float64
 8   ingcor      12815 non-null  float64
 9   gasmon      12815 non-null  float64
 10  gascor      12815 non-null  float64
 11  tam_hog     12815 non-null  int64  
 12  hombres     12815 non-null  int64  
 13  mujeres     12815 non-null  int64  
 14  tot_resi    12815 non-null  int64  
 15  tot_hom     12815 non-null  int64  
 16  tot_muj     12815 non-null  int64  
 17  ingtot      12815 non-null  float64
 18  pertot      12815 non-null  float64
 19  eromon      12815 non-nul

In [80]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60353 entries, 0 to 60352
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   folio       60353 non-null  object
 1   num_ren     60353 non-null  object
 2   parentesco  60353 non-null  object
 3   sexo        60353 non-null  object
 4   edad        60353 non-null  int64 
 5   per_ing     60353 non-null  object
 6   leer_esc    60353 non-null  object
 7   asis_esc_f  60353 non-null  object
 8   tipo_esc    60353 non-null  object
 9   ed_formal   60353 non-null  object
 10  ed_tecnica  60353 non-null  object
 11  trabajo     60353 non-null  object
 12  no_trabajo  60353 non-null  object
 13  n_empleos   60353 non-null  int64 
 14  ocupacion   60353 non-null  object
 15  oc_donde    60353 non-null  object
 16  rama        60353 non-null  object
 17  ra_donde    60353 non-null  object
 18  nom_empr    60353 non-null  object
 19  posicion    60353 non-null  object
 20  hrs_se

In [81]:
years = ['1994', '1996']
dfs = []

for year in years:
    print('-------------------------------------')
    print('Making year ' + year)
    os.chdir(raw + year)
    main = Dbf5('main.dbf').to_dataframe()
    population = Dbf5('population.dbf').to_dataframe()

    print('Number of houses:', len(main))
    print('Number of people:', len(population))

    main.columns = [x.lower() for x in main.columns]
    population.columns = [x.lower() for x in population.columns]
     
    main = main.rename(columns = rename_main)
    population = population.rename(columns = rename_population)

    keep_main = list(set(rename_main.values()) & set (main.columns))
    keep_population = list(set(rename_population.values()) & set (population.columns))

    main = main[keep_main]
    population = population[keep_population]

    population['relationship_to_hhm'] = population['relationship_to_hhm'].astype('int')
    population['is_household_head'] = population['relationship_to_hhm'].apply(lambda x : x in hhm_ids[year])

    population = population.query('is_household_head == True')
    print('Number of household heads', len(population))
    population['education'] = (population['education']
                               .astype(int)
                               .apply(lambda x : education[year][str(x)])
                               .astype(str))

    df = main.merge(population, on='hhid')

    # add year indicator
    df['Y'] = int(year)
    dfs.append(df)

-------------------------------------
Making year 1994
Number of houses: 12815
Number of people: 60353
Number of household heads 12815
-------------------------------------
Making year 1996
Number of houses: 14042
Number of people: 64916
Number of household heads 14042


In [82]:
# concatenate to build total dataset over each year 
df = pd.concat(dfs).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26857 entries, 0 to 26856
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   consumption_current         26857 non-null  float64
 1   location_size               26857 non-null  object 
 2   hhid                        26857 non-null  object 
 3   consumption_food            26857 non-null  float64
 4   income_current              26857 non-null  float64
 5   hh_size                     26857 non-null  int64  
 6   consumption_total           26857 non-null  float64
 7   consumption_personal        26857 non-null  float64
 8   consumption_clothing        26857 non-null  float64
 9   income_business             26857 non-null  float64
 10  income_salary               26857 non-null  float64
 11  income_transfers            26857 non-null  float64
 12  income_other                26857 non-null  float64
 13  weight                      268

# Gender

In [83]:
df['sex'] = df['sex'].replace({'1' : 'Male', '2':'Female'})
df.sex.value_counts(dropna=False)

sex
Male      22899
Female     3958
Name: count, dtype: int64

In [84]:
df['location'] = df.location.str[:2].astype(int)
df['zipcode'] = df['location'].copy()
df['location'] = df['location'].astype(str).replace(rename_location)
df['location'].value_counts()

location
Mexico City            15718
Coahuila                2794
Chiapas                 1835
X                       1731
Baja California         1707
Aguascalientes          1044
Chihuahua               1037
Campeche                 511
Baja California Sur      480
Name: count, dtype: int64

# Normalizing Income and Consumption

In [85]:
income_sources = ['income_salary', 'income_business', 'income_transfers']
consumption_sources = ['consumption_food', 'consumption_personal', 'consumption_clothing']

df['income'] = df[income_sources].sum(axis=1)
df['consumption'] = df[consumption_sources].sum(axis=1)

In [86]:
"""Normalization
This section normalizes income and consumption measures by two factors:
1. Household size
2. The price level at the time, given by the cpi
"""
df = df.merge(cpi, on='Y', how='left')
for column in df.columns:
    if 'income' in column or 'consumption' in column:
        df[column]  = df[column] / df['cpi']
        df[column] = df[column] / df['hh_size']

In [87]:
# remove households that do not consume anything
len_init = len(df)
df = df[df.consumption > 0]
len_finish = len(df)
print('Lost due to zero consumption:', 100*(len_init - len_finish) / len_init, '%')

len_init = len(df)
df = df[df.income > 0]
len_finish = len(df)
print('Lost due to zero income:', 100*(len_init - len_finish) / len_init, '%')

Lost due to zero consumption: 0.43564061510965485 %
Lost due to zero income: 0.512341062079282 %


In [88]:
q_999 = lambda x : np.quantile(x, 0.999)

len_init = len(df)
df = df[df.consumption <= df.groupby('Y', as_index=False)['consumption'].transform(q_999)]
len_final = len(df)
print(f'Lost due to consumption:{100*(len_init - len_final) / len_init :.2f} %')

len_init = len(df)
df = df[df.income <= df.groupby('Y', as_index=False)['income'].transform(q_999)]
len_final = len(df)
print(f'Lost due to income:{100*(len_init - len_final) / len_init :.2f} %')

Lost due to consumption:0.10 %
Lost due to income:0.10 %


In [89]:
df['log_income'] = np.log(df['income'])
df['log_consumption'] = np.log(df['consumption'])

# Education

In [90]:
df['education'] = pd.Categorical(df['education'], categories=['Low', 'Medium', 'High'], ordered=True)
df['education'] = df['education'].cat.reorder_categories(['Low', 'Medium', 'High'])
df.education.value_counts(dropna=False)

education
Low       18095
Medium     6028
High       2426
Name: count, dtype: int64

# Controlling for age

In [91]:
# remove household heads out of age bounds
len_init = len(df)
df = (df.query('age>=25')
      .query('age<=60'))
len_finish = len(df)
print('Lost due to age:', (len_init - len_finish) / len_init, '%')

Lost due to age: 0.23164714301856945 %


In [92]:
os.chdir(interim)
df.to_csv('main.csv', index=False)